In [ ]:
import numpy as np
import cupy as cp


import util
import NN as NN

from sklearn.metrics import accuracy_score

from datetime import datetime 


def Eval(ts_y,ts_x):
    def predict(X):
        total = cp.zeros((X.shape[0],10),dtype=cp.float32)
        for i in range(X.shape[0] // batch_size):
            X0 = X[i * batch_size : (i + 1) * batch_size]
            for Layer in sequential:
                X0 = Layer.Forward(X0)
            total[i * batch_size : (i + 1) * batch_size] = X0
        return total
    pd = predict(ts_x)
    pdinx = pd.get().argmax(axis=1)
    return "\tAccuracy_score: " + str(accuracy_score(ts_y, pdinx))

In [ ]:
tr_x,tr_y,ts_x,ts_y= util.load_torchMnist()

In [ ]:
batch_size = 100
 
loss = NN.MSEloss(10)

l1_size=800

l2_size=300

lr = 1

In [ ]:
sequential = [
        NN.Linear(batch_size, 784, l1_size),
        NN.Sigmoid(batch_size, l1_size),
        NN.Linear(batch_size, l1_size, l2_size),
        NN.Sigmoid(batch_size, l2_size),
        NN.Linear(batch_size, l2_size, 10),
        NN.Sigmoid(batch_size, 10)
]

In [ ]:
start_time = datetime.now() 
print("[INFO]  Train started!\t", start_time)

for ep in range(30):
    for i in range(tr_x.shape[0] // batch_size):
        X = tr_x[i * batch_size : (i + 1) * batch_size]
        Y = tr_y[i * batch_size : (i + 1) * batch_size]
        
        for Layer in sequential:
            X = Layer.Forward(X)

        dY = loss.Backward(X, Y)

        for Layer in reversed(sequential):
            dY = Layer.Backward(dY)

        for Layer in sequential:
            Layer.Update(lr=lr)

    if ep % 1 == 0:
        print(
            f"[{ep}] \t--> {Eval(ts_y,ts_x)}  \t\t/ loss mini-batch : {loss.LossVal:.6f}"
        )

time_elapsed = (datetime.now() - start_time )
print("[INFO]  Train finished!\t", datetime.now() , "\t Run time: ",time_elapsed)
